In [2]:
import faiss

Loading faiss with AVX2 support.


In [3]:
fname = '....'
idx = faiss.read_index(fname)

# Explore a loaded index

In [4]:
idx

<faiss.swigfaiss_avx2.IndexPreTransform; proxy of <Swig Object of type 'faiss::IndexPreTransform *' at 0x7fc0c46a7ea0> >

So it's an `IndexPreTransform` that wraps another index. To see what's inside, use `downcast_index`.

In [5]:
faiss.downcast_index(idx.index)

<faiss.swigfaiss_avx2.IndexIVFScalarQuantizer; proxy of <Swig Object of type 'faiss::IndexIVFScalarQuantizer *' at 0x7fc0bc08c3c0> >

An IVFScalarQuantizer. How many centroids? Is it trained? What does the quantizer look like?  

In [9]:
faiss.downcast_index(idx.index).nlist

1048576

In [10]:
faiss.downcast_index(idx.index).is_trained

True

In [11]:
faiss.downcast_index(idx.index).quantizer

<faiss.swigfaiss_avx2.Index; proxy of <Swig Object of type 'faiss::Index *' at 0x7fc0bc08c2d0> >

Obviously it's an index. But what type of index? 

In [12]:
faiss.downcast_index(faiss.downcast_index(idx.index).quantizer)

<faiss.swigfaiss_avx2.IndexFlat; proxy of <Swig Object of type 'faiss::IndexFlat *' at 0x7fc0bc11eb10> >

# Moving to GPU

In [13]:
co = faiss.GpuMultipleClonerOptions()
co.shard = True
idx_gpu = faiss.index_cpu_to_all_gpus(idx, co)


In [14]:
idx_gpu

<faiss.swigfaiss_avx2.IndexPreTransform; proxy of <Swig Object of type 'faiss::IndexPreTransform *' at 0x7fc0bc0921e0> >

Again behind a PreTransform (on CPU). Let's see what's inside that.

In [15]:
faiss.downcast_index(idx_gpu.index)

<faiss.swigfaiss_avx2.IndexShards; proxy of <Swig Object of type 'faiss::IndexShardsTemplate< faiss::Index > *' at 0x7fc0bc08c540> >

Here are the shards. What does each shard contain? 

In [16]:
faiss.downcast_index(idx_gpu.index).at(0)

<faiss.swigfaiss_avx2.Index; proxy of <Swig Object of type 'faiss::Index *' at 0x7fc0bc0920c0> >

Let's downcast once more  

In [17]:
faiss.downcast_index(faiss.downcast_index(idx_gpu.index).at(0))

<faiss.swigfaiss_avx2.GpuIndexIVFScalarQuantizer; proxy of <Swig Object of type 'faiss::gpu::GpuIndexIVFScalarQuantizer *' at 0x7fc041d9dab0> >

So here is the GPU Scalar Quantizer. How many inverted lists? 

In [18]:
faiss.downcast_index(faiss.downcast_index(idx_gpu.index).at(0)).nlist

1048576